In [1]:
import pandas as pd
import numpy as np
import os
import re
from pandas_profiling import ProfileReport
from tqdm.auto import tqdm

tqdm.pandas()
workers = os.cpu_count()

# Data Cleaning

In [2]:
all_rounds = pd.read_csv('data/scraped/parsed/all_rounds_merged.csv')
all_splits = pd.read_csv('data/scraped/parsed/all_splits_merged.csv')

C:\Users\ahale\Anaconda3\envs\shorttrack\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\ahale\Anaconda3\envs\shorttrack\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (1,4,7,10,19,22,29,31,32,35,38,41,44,47,50,53,56,59,62) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
def parse_time_string(s):
    if '.' in str(s):
        m = re.match(r'(?P<days>[-\d]+) day[s]*, (?P<hours>\d+):(?P<minutes>\d+):(?P<seconds>\d[\.\d+]*)', str(s))
        if m is not None:
            return pd.Timedelta(**{key: float(val) for key, val in m.groupdict().iteritems()})
    return s

In [4]:
def indicate_multiple_of_same_distance(event_name):
    instance_of_event_in_competition = np.nan
    if '(1)' in event_name:
        return 1
    elif '(2)' in event_name:
        return 2
    return -1

In [5]:
all_rounds["instance_of_event_in_competition"] = all_rounds['event'].apply(lambda event: indicate_multiple_of_same_distance(event))
all_splits["instance_of_event_in_competition"] = all_splits['event'].apply(lambda event: indicate_multiple_of_same_distance(event))

In [6]:
event_name_cleanup = {"500M": "500m",
                      "500m(1)": "500m",
                      "500m(2)": "500m",
                      "1000M": "1000m",
                      "1000m(1)": "1000m",
                      "1000m(2)": "1000m",
                      "1500M": "1500m",
                      "1500m(1)": "1500m",
                      "1500m(2)": "1500m",
                      "1,000m": "1000m",
                      "3000M": "3000m",
                      "1,500m": "1500m",
                      "3000MRelay": "3000mRelay",
                      "5000M": "5000m",
                      "5000MRelay": "5000mRelay",
                      "3,000mRelay": "3000mRelay",
                      "5,000mRelay": "5000mRelay",
                      "2000MRelay": "2000mRelay"
                     }
all_rounds["event"].replace(to_replace=event_name_cleanup, inplace=True)
all_splits["event"].replace(to_replace=event_name_cleanup, inplace=True)

In [7]:
all_rounds.drop("Relay Team", axis=1, inplace=True)

In [8]:
all_rounds["Results"] = all_rounds["Results"].apply(lambda result: parse_time_string(result))

In [9]:
for col in all_splits.columns:
    if 'TIME' in col:
        all_splits[col] = all_splits[col].apply(lambda t: parse_time_string(t))

# Merge Rounds and Splits into one DataFrame

In [10]:
individual_races = all_rounds.groupby(['season', 'competition', 'event', 'gender', 'round', 'race', 'instance_of_event_in_competition'])
rounds_splits_df = all_rounds.copy()

for race_details, athlete_race_data in tqdm(individual_races):
    athlete_indices = athlete_race_data.index
    
    laps = all_splits[(all_splits['season'] == race_details[0]) & 
                      (all_splits['competition'] == race_details[1]) & 
                      (all_splits['event'] == race_details[2]) & 
                      (all_splits['gender'] == race_details[3]) & 
                      (all_splits['round'] == race_details[4]) & 
                      (all_splits['race'] == race_details[5]) &
                      (all_splits['instance_of_event_in_competition'] == race_details[6])]
    
    rounds_splits_df.loc[athlete_indices, 'laps_of_split_data'] = laps.shape[0]

    if laps.shape[0] > 1:
        for athlete_index in athlete_indices:
            athlete_start_position = f'START_POS_{rounds_splits_df.loc[athlete_index, "Start Pos."]}'

            if f'{athlete_start_position} POSITION' in laps.columns:
                for lap_number, (lap_index, lap_data) in enumerate(laps.iterrows()):
                    rounds_splits_df.loc[athlete_index, f'lap_{lap_number + 1}_position'] = lap_data[f'{athlete_start_position} POSITION']
                    rounds_splits_df.loc[athlete_index, f'lap_{lap_number + 1}_laptime'] = lap_data[f'{athlete_start_position} LAP TIME']
                    rounds_splits_df.loc[athlete_index, f'lap_{lap_number + 1}_elapsedtime'] = lap_data[f'{athlete_start_position} ELAPSED TIME']

# Save

In [11]:
all_rounds.to_csv('data/scraped/cleaned/all_rounds.csv', index=False)
all_splits.to_csv('data/scraped/cleaned/all_splits.csv', index=False)
rounds_splits_df.to_csv('data/scraped/cleaned/rounds_splits.csv', index=False)

# Profile

In [ ]:
ProfileReport(rounds_splits_df, title="Races with Splits - Profile Report")